In [1]:
 !cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 85
model name	: Intel(R) Xeon(R) CPU @ 2.00GHz
stepping	: 3
microcode	: 0xffffffff
cpu MHz		: 2000.184
cache size	: 39424 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm mpx avx512f avx512dq rdseed adx smap clflushopt clwb avx512cd avx512bw avx512vl xsaveopt xsavec xgetbv1 xsaves arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa mmi

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [3]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-gstcfrfx
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-gstcfrfx
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4294 sha256=a20c952b729bc0661051fecc5a1288744f6dad8361e076a81c60329d43c030b2
  Stored in directory: /tmp/pip-ephem-wheel-cache-8h9evkqc/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [4]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [5]:

%%cuda --name cuda_SSSP_BellmanFord.cu

#include <cuda_runtime.h>
#include <device_launch_parameters.h>
#include <iostream>
#include <stdio.h>
#include <stdlib.h>
#include <limits.h>

#define MAX_INT INT_MAX
#define BLOCKSIZE 128
#define GRIDSIZE 256
#define SEED_VALUE 128
#define MAX_VERTICES 32768
#define NUM_PREDECESSORS 3
#define MAX_WEIGHT 3
#define MAX_EDGES (MAX_VERTICES * NUM_PREDECESSORS)

__global__ void bellmanFord(int *dev_edges, int *dev_weights, int *dev_distances, int num_vertices, int num_edges) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;

    if (tid < num_vertices) {
        for (int i = 0; i < num_edges; ++i) {
            int source = dev_edges[i * 3];
            int destination = dev_edges[i * 3 + 1];
            int weight = dev_weights[i];

            if (dev_distances[source] != MAX_INT && dev_distances[source] + weight < dev_distances[destination]) {

                atomicMin(&dev_distances[destination], dev_distances[source] + weight);
            }
        }
    }
}

int main() {


    int edges[MAX_EDGES][3];  // 3 columns: source, target, weight for each edge
    int distances[MAX_VERTICES];
    int weights[MAX_EDGES];
    int num_vertices = MAX_VERTICES;
    int num_edges = 0;

    // Initialize distances array
    distances[0] = 0;
    for (int i = 1; i < MAX_VERTICES; ++i) {
        distances[i] = INT_MAX;  // Assuming distances are initially set to infinity
    }

        /* seed -- create random graph */
    srand(SEED_VALUE);

    int (*h_predecessors)[MAX_VERTICES][2];
    int predBytes = NUM_PREDECESSORS * sizeof *h_predecessors;
    h_predecessors = (int (*)[MAX_VERTICES][2])malloc(predBytes);

    /* initialize random graph (plus a linear path beginning to end) */
    for (int i = 0; i < MAX_VERTICES; ++i) {
        h_predecessors[0][i][0] = i-1;
        h_predecessors[0][i][1] = 1;


        int j = 1;
        while (j < min(MAX_VERTICES-1,NUM_PREDECESSORS)) {
            int pred = rand() % MAX_VERTICES;
            int weight = (MAX_WEIGHT > 1) ? (1 + (rand() % (MAX_WEIGHT - 1))) : 1;

            // no self-loop
            if (pred == i) {
                continue;
            }

            // check it was not already used
            int found = 0;
            for (int k = 0; k < j; ++k) {
                if (h_predecessors[k][i][0] == pred) {
                    found = 1;
                }
            }
            if (found) {
                continue;
            }

            h_predecessors[j][i][0] = pred;
            h_predecessors[j][i][1] = weight;
            ++j;
        }

        // sentinel
        if (j < NUM_PREDECESSORS) {
            h_predecessors[j][i][0] = -1;
        }
    }


    int edgeIndex = 0;
    for (int i = 0; i < MAX_VERTICES; ++i) {
    // Linear path from i-1 to i with weight 1
      edges[edgeIndex][0] = i - 1;  // source
      edges[edgeIndex][1] = i;      // target
      edges[edgeIndex][2] = 1;      // weight
      weights[edgeIndex] = 1;       // weight
      ++edgeIndex;

    // Add random incoming edges for each vertex
    int j = 1;
    while (j < min(MAX_VERTICES - 1, NUM_PREDECESSORS) && h_predecessors[j][i][0] != -1) {
        edges[edgeIndex][0] = h_predecessors[j][i][0];  // source
        edges[edgeIndex][1] = i;                          // target
        edges[edgeIndex][2] = h_predecessors[j][i][1];   // weight
        weights[edgeIndex] = h_predecessors[j][i][1];    // weight
        ++j;
        ++edgeIndex;
    }

  }

    num_edges = edgeIndex;

        // Start tracking the execution time of Bellman Ford on GPU
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    std::cout << "Running Bellman Ford on GPU" << std::endl;
    cudaEventRecord(start, 0);

    int *dev_edges, *dev_weights, *dev_distances;

    cudaMalloc((void**)&dev_edges, num_edges * 3 * sizeof(int));
    cudaMalloc((void**)&dev_weights, num_edges * sizeof(int));
    cudaMalloc((void**)&dev_distances, num_vertices * sizeof(int));


    cudaMemcpy(dev_edges, edges, num_edges * 3 * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dev_weights, weights, num_edges * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dev_distances, distances, num_vertices * sizeof(int), cudaMemcpyHostToDevice);



    bellmanFord<<<GRIDSIZE, BLOCKSIZE>>>(dev_edges, dev_weights, dev_distances, num_vertices, num_edges);


    cudaMemcpy(distances, dev_distances, num_vertices * sizeof(int), cudaMemcpyDeviceToHost);

    // Stop tracking the execution time of Bellman Ford on GPU
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
    std::cout << "Completed Execution of Bellman Ford on GPU" << std::endl;
    float elapsedTime;
    cudaEventElapsedTime(&elapsedTime, start, stop);

    // Print the distances
     for (int i = 0; i < num_vertices; ++i) {
       std::cout << "Distance to vertex " << i << " is: " << distances[i] << std::endl;
    }

    std::cout << "Time elapsed for " << num_vertices << " vertices: " << elapsedTime << " milliseconds\n";

    cudaFree(dev_edges);
    cudaFree(dev_weights);
    cudaFree(dev_distances);

  return 0;
}

'File written in /content/src/cuda_SSSP_BellmanFord.cu'

In [6]:

!nvcc -arch=sm_75 -o "/content/src/cuda_SSSP_BellmanFord.o" /content/src/cuda_SSSP_BellmanFord.cu

In [7]:
!chmod 755 /content/src/cuda_SSSP_BellmanFord.o
!/content/src/cuda_SSSP_BellmanFord.o

Streaming output truncated to the last 5000 lines.
Distance to vertex 27769 is: 20
Distance to vertex 27770 is: 19
Distance to vertex 27771 is: 20
Distance to vertex 27772 is: 21
Distance to vertex 27773 is: 15
Distance to vertex 27774 is: 16
Distance to vertex 27775 is: 17
Distance to vertex 27776 is: 18
Distance to vertex 27777 is: 19
Distance to vertex 27778 is: 20
Distance to vertex 27779 is: 21
Distance to vertex 27780 is: 11
Distance to vertex 27781 is: 12
Distance to vertex 27782 is: 13
Distance to vertex 27783 is: 14
Distance to vertex 27784 is: 15
Distance to vertex 27785 is: 16
Distance to vertex 27786 is: 17
Distance to vertex 27787 is: 17
Distance to vertex 27788 is: 17
Distance to vertex 27789 is: 18
Distance to vertex 27790 is: 19
Distance to vertex 27791 is: 19
Distance to vertex 27792 is: 17
Distance to vertex 27793 is: 18
Distance to vertex 27794 is: 19
Distance to vertex 27795 is: 14
Distance to vertex 27796 is: 15
Distance to vertex 27797 is: 16
Distance to vertex 27

In [8]:
%%cuda --name cpu_SSSP_BellmanFord.cu

#include <iostream>
#include <cstdlib>
#include <ctime>
#include <climits>

#define MAX_INT INT_MAX
#define MAX_VERTICES 32768
#define NUM_PREDECESSORS 3
#define MAX_WEIGHT 3
#define MAX_EDGES (MAX_VERTICES * NUM_PREDECESSORS)
#define SEED_VALUE 128

void bellmanFord(int edges[MAX_EDGES][3], int weights[MAX_EDGES], int distances[MAX_VERTICES], int num_vertices, int num_edges) {
#pragma omp parallel for
for (int i = 0; i < num_vertices; ++i) {
    for (int j = 0; j < num_edges; j += 2) {
        int source1 = edges[j][0];
        int destination1 = edges[j][1];
        int weight1 = weights[j];

        int source2 = edges[j + 1][0];
        int destination2 = edges[j + 1][1];
        int weight2 = weights[j + 1];

        if (distances[source1] != MAX_INT && distances[source1] + weight1 < distances[destination1]) {
            distances[destination1] = distances[source1] + weight1;
        }

        if (distances[source2] != MAX_INT && distances[source2] + weight2 < distances[destination2]) {
            distances[destination2] = distances[source2] + weight2;
        }
    }
}

}



int main() {
    int edges[MAX_EDGES][3];  // 3 columns: source, target, weight for each edge
    int distances[MAX_VERTICES];
    int weights[MAX_EDGES];
    int num_vertices = MAX_VERTICES;
    int num_edges = 0;
    // Initialize distances array
    distances[0] = 0;
    for (int i = 1; i < MAX_VERTICES; ++i) {
        distances[i] = INT_MAX;  // Assuming distances are initially set to infinity
    }

        /* seed -- create random graph */
    srand(SEED_VALUE);

    int (*h_predecessors)[MAX_VERTICES][2];
    int predBytes = NUM_PREDECESSORS * sizeof *h_predecessors;
    h_predecessors = (int (*)[MAX_VERTICES][2])malloc(predBytes);

    /* initialize random graph (plus a linear path beginning to end) */
    for (int i = 0; i < MAX_VERTICES; ++i) {
        h_predecessors[0][i][0] = i-1;
        h_predecessors[0][i][1] = 1;


        int j = 1;
        while (j < min(MAX_VERTICES-1,NUM_PREDECESSORS)) {
            int pred = rand() % MAX_VERTICES;
            int weight = (MAX_WEIGHT > 1) ? (1 + (rand() % (MAX_WEIGHT - 1))) : 1;

            // no self-loop
            if (pred == i) {
                continue;
            }

            // check it was not already used
            int found = 0;
            for (int k = 0; k < j; ++k) {
                if (h_predecessors[k][i][0] == pred) {
                    found = 1;
                }
            }
            if (found) {
                continue;
            }

            h_predecessors[j][i][0] = pred;
            h_predecessors[j][i][1] = weight;
            ++j;
        }

        // sentinel
        if (j < NUM_PREDECESSORS) {
            h_predecessors[j][i][0] = -1;
        }
    }

    int edgeIndex = 0;
    for (int i = 0; i < MAX_VERTICES; ++i) {
    // Linear path from i-1 to i with weight 1
      edges[edgeIndex][0] = i - 1;  // source
      edges[edgeIndex][1] = i;      // target
      edges[edgeIndex][2] = 1;      // weight
      weights[edgeIndex] = 1;       // weight
      ++edgeIndex;

    // Add random incoming edges for each vertex
    int j = 1;
    while (j < min(MAX_VERTICES - 1, NUM_PREDECESSORS) && h_predecessors[j][i][0] != -1) {

        edges[edgeIndex][0] = h_predecessors[j][i][0];  // source
        edges[edgeIndex][1] = i;                          // target
        edges[edgeIndex][2] = h_predecessors[j][i][1];   // weight
        weights[edgeIndex] = h_predecessors[j][i][1];    // weight
        ++j;
        ++edgeIndex;
    }
  }
    num_edges = edgeIndex;
        // Start tracking the execution time of Bellman Ford on CPU
    clock_t start, stop;
    std::cout << "Running Bellman Ford on CPU" << std::endl;
    start = clock();

    bellmanFord(edges, weights, distances, num_vertices, num_edges);

    // Stop tracking the execution time of Bellman Ford on CPU
    stop = clock();
    std::cout << "Completed Execution of Bellman Ford on CPU" << std::endl;
    double elapsedTime = double(stop - start) / CLOCKS_PER_SEC * 1000.0;  // Convert to milliseconds

    // Print the distances
    for (int i = 0; i < num_vertices; ++i) {
        std::cout << "Distance to vertex " << i << " is: " << distances[i] << std::endl;
    }

    std::cout << "Time elapsed for " << num_vertices << " vertices: " << elapsedTime << " milliseconds\n";

    return 0;
}



'File written in /content/src/cpu_SSSP_BellmanFord.cu'

In [9]:
!nvcc -arch=sm_75 -o "/content/src/cpu_SSSP_BellmanFord.o" /content/src/cpu_SSSP_BellmanFord.cu

In [10]:
!chmod 755 /content/src/cpu_SSSP_BellmanFord.o
!/content/src/cpu_SSSP_BellmanFord.o

Streaming output truncated to the last 5000 lines.
Distance to vertex 27769 is: 14
Distance to vertex 27770 is: 13
Distance to vertex 27771 is: 13
Distance to vertex 27772 is: 14
Distance to vertex 27773 is: 14
Distance to vertex 27774 is: 15
Distance to vertex 27775 is: 14
Distance to vertex 27776 is: 14
Distance to vertex 27777 is: 7
Distance to vertex 27778 is: 8
Distance to vertex 27779 is: 9
Distance to vertex 27780 is: 10
Distance to vertex 27781 is: 11
Distance to vertex 27782 is: 12
Distance to vertex 27783 is: 13
Distance to vertex 27784 is: 12
Distance to vertex 27785 is: 11
Distance to vertex 27786 is: 12
Distance to vertex 27787 is: 13
Distance to vertex 27788 is: 11
Distance to vertex 27789 is: 12
Distance to vertex 27790 is: 13
Distance to vertex 27791 is: 13
Distance to vertex 27792 is: 10
Distance to vertex 27793 is: 11
Distance to vertex 27794 is: 12
Distance to vertex 27795 is: 13
Distance to vertex 27796 is: 13
Distance to vertex 27797 is: 12
Distance to vertex 27798